In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Leer el CSV correctamente ahora que el archivo está limpio
datos = pd.read_csv("Shark_Attacks.csv", parse_dates=["date"])

# Verifica que se cargó bien
print(datos.head())

# Rellenar valores vacíos en algunas columnas
datos["species"] = datos["species"].fillna("Unknown")
datos["fatal_y_n"] = datos["fatal_y_n"].map({"Y": "Fatal", "N": "Non-fatal"})
datos["type"] = datos["type"].fillna("Unknown")

# Obtener año y mes
datos["year"] = datos["date"].dt.year
datos["month"] = datos["date"].dt.month

# Clasificar hora del día en 4 categorías
def clasificar_hora(t):
    if pd.isna(t):
        return "Unknown"
    try:
        hora = int(str(t).split(":")[0])
    except:
        t = str(t).lower()
        if "morning" in t:
            hora = 9
        elif "afternoon" in t:
            hora = 14
        elif "evening" in t:
            hora = 18
        elif "night" in t:
            hora = 22
        else:
            return "Unknown"
    if hora >= 5 and hora < 12:
        return "Morning"
    elif hora >= 12 and hora < 17:
        return "Afternoon"
    elif hora >= 17 and hora < 21:
        return "Evening"
    else:
        return "Night"

datos["time_cat"] = datos["time"].apply(clasificar_hora)

# Convertir país a continente de forma muy básica (solo algunos países conocidos)
def pais_a_continente_simple(pais):
    if pais in ["United States", "USA", "Mexico", "Bahamas"]:
        return "North America"
    elif pais in ["Australia", "New Zealand"]:
        return "Oceania"
    elif pais in ["South Africa", "Mozambique", "Reunion"]:
        return "Africa"
    elif pais in ["Brazil", "Argentina"]:
        return "South America"
    elif pais in ["Spain", "Italy", "France"]:
        return "Europe"
    else:
        return "Other"

datos["continent"] = datos["country"].apply(pais_a_continente_simple)

# Gráfico 1 - Especies con más ataques
top_especies = datos["species"].value_counts().head(10)
plt.figure()
top_especies.plot(kind="bar")
plt.title("Top 10 especies con más ataques")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Gráfico 2 - Ataques fatales y no fatales por especie
tabla1 = datos.pivot_table(index="species", columns="fatal_y_n", aggfunc="size", fill_value=0)
tabla1 = tabla1.loc[top_especies.index]
tabla1.plot(kind="bar", stacked=True)
plt.title("Ataques fatales y no fatales por especie")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Gráfico 3 - Países con más ataques
top_paises = datos["country"].value_counts().head(15)
top_paises.plot(kind="bar")
plt.title("Top 15 países con más ataques")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Gráfico 4 - Ataques fatales/no fatales por país
tabla2 = datos.pivot_table(index="country", columns="fatal_y_n", aggfunc="size", fill_value=0)
tabla2 = tabla2.loc[top_paises.index[:10]]
tabla2.plot(kind="bar", stacked=True)
plt.title("Ataques fatales y no fatales por país")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Gráfico 5 - Ataques provocados y no provocados (total y fatales)
tipo_total = datos["type"].value_counts()
tipo_fatal = datos[datos["fatal_y_n"] == "Fatal"]["type"].value_counts()
plt.bar(tipo_total.index, tipo_total.values, label="Total")
plt.bar(tipo_fatal.index, tipo_fatal.values, label="Fatales")
plt.title("Provocados vs No provocados")
plt.legend()
plt.tight_layout()
plt.show()

# Gráfico 6 - Actividades más comunes durante el ataque
top_actividades = datos["activity"].value_counts().head(10)
top_actividades.plot(kind="bar")
plt.title("Top 10 actividades durante ataques")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Gráfico 7 - Ataques por hora del día
horas = datos["time_cat"].value_counts()
horas.plot(kind="bar")
plt.title("Ataques por hora del día")
plt.tight_layout()
plt.show()

# Gráfico 8 - Hora del día vs especie (top 5 especies)
top5_especies = top_especies.index[:5]
filtro = datos[datos["species"].isin(top5_especies)]
tabla_horas = filtro.pivot_table(index="time_cat", columns="species", aggfunc="size", fill_value=0)
tabla_horas.plot(kind="bar")
plt.title("Hora del día vs Especie (Top 5)")
plt.xticks(rotation=0)
plt.legend()
plt.tight_layout()
plt.show()

# Gráfico 9 - Ataques por mes
mensual = datos.groupby(pd.Grouper(key="date", freq="M")).size()
mensual.plot()
plt.title("Ataques por mes")
plt.tight_layout()
plt.show()

# Gráfico 10 - Ataques por continente
continente = datos["continent"].value_counts()
continente.plot(kind="bar")
plt.title("Ataques por continente")
plt.tight_layout()
plt.show()

# Gráfico 11 - Ataques por año
anios = datos["year"].value_counts().sort_index()
plt.plot(anios.index, anios.values, marker="o")
plt.title("Ataques por año")
plt.tight_layout()
plt.show()
